# Table view and model

In lesson `18.QTableWidget`, we build the view row by row. In lesson `03.QTableView`, we use model to convert raw data to pyqt data model, then use the view to render the data model automatically.

In this lesson, we will see how to sort rows, with a user input, we can also filter rows

## 1 Simple example with sorting

In below example,
1. we build a custom table model which takes a two dimension list as source data.
2. We create a **proxy model(QSortFilterProxyModel())** which handles the sorting

First the proxy model needs the source model to get the data. Then we set sorting logic.
```python
# connect proxy model with source model
self.proxy_model.setSourceModel(self.model)

# set proxy model sorting order by using the value of first column with index 0
self.proxy_model.sort(0, Qt.SortOrder.AscendingOrder)

```

In [ ]:
import sys
from typing import Any

from PyQt6.QtWidgets import QApplication, QWidget, QTableView, QMainWindow, QVBoxLayout, QLineEdit
from PyQt6.QtCore import Qt, QSortFilterProxyModel, QAbstractTableModel


class MyTableModel(QAbstractTableModel):
    """
    This class is a custom table model, we just implement the three mandatory method (data, rowCount, columnCount)
    """
    def __init__(self, data):
        super().__init__()
        self._data = data

    def data(self, index, role):
        if role == Qt.ItemDataRole.DisplayRole:
            # See below for the nested-list data structure.
            # .row() indexes into the outer list,
            # .column() indexes into the sub-list
            return self._data[index.row()][index.column()]

    def rowCount(self, index:Any):
        # The length of the outer list.
        return len(self._data)

    def columnCount(self, index):
        # The following takes the first sub-list, and returns
        # the length (only works if all rows are an equal length)
        return len(self._data[0])


class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        # create a table view
        self.table = QTableView()

        data = [
            [5, 9, 2],
            [4, "hello", 0],
            [1, 5, 0],
            [2, 3, "what"],
            [3, 8, 9],
        ]

        # create the source model
        self.model = MyTableModel(data)
        # create a proxy model for filtering and sorting
        self.proxy_model = QSortFilterProxyModel()

        # connect proxy model with source model
        self.proxy_model.setSourceModel(self.model)

        # set proxy model sorting order by using the value of first column with index 0
        self.proxy_model.sort(0, Qt.SortOrder.AscendingOrder)

        # connect the view with proxy model
        self.table.setModel(self.proxy_model)

        layout = QVBoxLayout()
        layout.addWidget(self.table)

        container = QWidget()
        container.setLayout(layout)

        self.setCentralWidget(container)


app = QApplication(sys.argv)
window = MainWindow()
window.show()
app.exec()

## 2 Simple example with filtering

In below example, we use `self.proxy_model.setFilterKeyColumn(-1)` to set which column value that we want to filter. Then we create a `QLineEdit` to get user input, when user change the text, a slot `search` will be called. In this slot we use `self.proxy_model.setFilterFixedString(text)` to filter the rows which has corresponding row.

In [ ]:
import sys
from typing import Any

from PyQt6.QtWidgets import QApplication, QWidget, QTableView, QMainWindow, QVBoxLayout, QLineEdit
from PyQt6.QtCore import Qt, QSortFilterProxyModel, QAbstractTableModel


class MyTableModel(QAbstractTableModel):
    """
    This class is a custom table model, we just implement the three mandatory method (data, rowCount, columnCount)
    """

    def __init__(self, data):
        super().__init__()
        self._data = data

    def data(self, index, role):
        if role == Qt.ItemDataRole.DisplayRole:
            # See below for the nested-list data structure.
            # .row() indexes into the outer list,
            # .column() indexes into the sub-list
            return self._data[index.row()][index.column()]

    def rowCount(self, index: Any):
        # The length of the outer list.
        return len(self._data)

    def columnCount(self, index):
        # The following takes the first sub-list, and returns
        # the length (only works if all rows are an equal length)
        return len(self._data[0])


class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        # create a table view
        self.table = QTableView()

        data = [
            [5, "hello_1", 2],
            [4, "hello_2", "88 bye" ],
            [1, 5, 0],
            [2, 3, "what"],
            [3, "this", "91Nice"],
        ]

        # create the source model
        self.model = MyTableModel(data)
        # create a proxy model for filtering and sorting
        self.proxy_model = QSortFilterProxyModel()
        # set search column index. for example 0 will be the first column, if we set -1, it will search all columns
        self.proxy_model.setFilterKeyColumn(-1)  # Search all columns.
        # connect proxy model with source model
        self.proxy_model.setSourceModel(self.model)

        # connect the view with proxy model
        self.table.setModel(self.proxy_model)

        # the search bar
        self.searchbar = QLineEdit()

        self.searchbar.textChanged.connect(self.search)


        layout = QVBoxLayout()

        layout.addWidget(self.searchbar)
        layout.addWidget(self.table)

        container = QWidget()
        container.setLayout(layout)

        self.setCentralWidget(container)

    def search(self, text):
        # You can choose the type of search by connecting to a different slot here.
        # see https://doc.qt.io/qt-6/qsortfilterproxymodel.html#public-slots
        # here we use setFilterFixedString (a string match)
        self.proxy_model.setFilterFixedString(text)
        # for example, we can use setFilterRegularExpression to take a regular expression
        # self.proxy_model.setFilterRegularExpression(text)


app = QApplication(sys.argv)
window = MainWindow()
window.show()
app.exec()


## 3. Filter with RegularExpression

In the above example, if we replace `self.proxy_model.setFilterFixedString(text)` by `self.proxy_model.setFilterRegularExpression(text)`. The user input can be a `QRegularExpression`, for more information, you can visit this [page](https://doc.qt.io/qt-6/qregularexpression.html)

Try to put `\d\d\w+` in the qline, you should see the row with `91Nice`. The regex means two digits followed by a sequence of alphabetic. Try to put `\d\d \w+`, you should see the row with `88 bye`. Because we add a `space` between digit and word.

## 4. Converting table view to list view by using proxyModel

In below example, we use proxy model to render the same source model with table view and list view

In [ ]:
import random
import math
from PyQt6 import QtCore
from PyQt6.QtCore import QSortFilterProxyModel, QIdentityProxyModel
from PyQt6.QtWidgets import QWidget, QHBoxLayout, QSplitter, QTableView, QListView, QApplication


class TableModel(QtCore.QAbstractTableModel):
    def __init__(self, data, columns, parent=None):
        super(TableModel, self).__init__(parent)
        self._columns = columns
        self._data = data[:]

    def rowCount(self, parent=QtCore.QModelIndex()):
        if parent.isValid() or self._columns == 0:
            return 0
        return math.ceil(len(self._data) * 1.0 / self._columns)

    def columnCount(self, parent=QtCore.QModelIndex()):
        if parent.isValid():
            return 0
        return self._columns

    def data(self, index, role=QtCore.Qt.ItemDataRole.DisplayRole):
        if not index.isValid():
            return
        if role == QtCore.Qt.ItemDataRole.DisplayRole:
            try:
                value = self._data[index.row() * self._columns + index.column()]
                return value
            except:
                pass


class Table2ListProxyModel(QIdentityProxyModel):
    def columnCount(self, parent=QtCore.QModelIndex()):
        return 1

    def rowCount(self, parent=QtCore.QModelIndex()):
        if parent.isValid():
            return 0
        return self.sourceModel().rowCount() * self.sourceModel().columnCount()

    def mapFromSource(self, sourceIndex):
        if sourceIndex.isValid() and sourceIndex.column() == 0 \
                and sourceIndex.row() < self.rowCount():
            r = sourceIndex.row()
            c = sourceIndex.column()
            row = c * sourceIndex.model().columnCount() + r
            return self.index(row, 0)
        return QtCore.QModelIndex()

    def mapToSource(self, proxyIndex):
        r = proxyIndex.row() / self.sourceModel().columnCount()
        c = proxyIndex.row() % self.sourceModel().columnCount()
        return self.sourceModel().index(int(r), c)

    def index(self, row, column, parent=QtCore.QModelIndex()):
        return self.createIndex(row, column)


class ListFilterProxyModel(QSortFilterProxyModel):
    def setThreshold(self, value):
        setattr(self, "threshold", value)
        self.invalidateFilter()

    def filterAcceptsRow(self, row, parent):
        if hasattr(self, "threshold"):
            ix = self.sourceModel().index(row, 0)
            val = ix.data()
            if val is None:
                return False
            return val < getattr(self, "threshold")
        return True


class List2TableProxyModel(QIdentityProxyModel):
    def __init__(self, columns=1, parent=None):
        super(List2TableProxyModel, self).__init__(parent)
        self._columns = columns

    def columnCount(self, parent=QtCore.QModelIndex()):
        return self._columns

    def rowCount(self, parent=QtCore.QModelIndex()):
        if parent.isValid():
            return 0
        return math.ceil(self.sourceModel().rowCount() / self._columns)

    def index(self, row, column, parent=QtCore.QModelIndex()):
        return self.createIndex(row, column)

    def data(self, index, role=QtCore.Qt.ItemDataRole.DisplayRole):
        r = index.row()
        c = index.column()
        row = r * self.columnCount() + c
        if row < self.sourceModel().rowCount():
            return super(List2TableProxyModel, self).data(index, role)

    def mapFromSource(self, sourceIndex):
        r = math.ceil(sourceIndex.row() / self.columnCount())
        c = sourceIndex.row() % self.columnCount()
        return self.index(r, c)

    def mapToSource(self, proxyIndex):
        if proxyIndex.isValid():
            r = proxyIndex.row()
            c = proxyIndex.column()
            row = r * self.columnCount() + c
            return self.sourceModel().index(row, 0)
        return QtCore.QModelIndex()


class Widget(QWidget):
    def __init__(self, parent=None):
        QWidget.__init__(self, parent)
        data = [8,7,6,1,2,3,4,5]

        layout = QHBoxLayout(self)
        splitter = QSplitter()
        layout.addWidget(splitter)

        tv = QTableView()
        lv = QListView()
        lvf = QListView()
        tvf = QTableView()

        model = TableModel(data, 3, self)
        proxy1 = Table2ListProxyModel(self)
        proxy1.setSourceModel(model)
        proxy2 = ListFilterProxyModel(self)
        proxy2.setSourceModel(proxy1)
        proxy2.setThreshold(5)
        proxy3 = List2TableProxyModel(3, self)
        proxy3.setSourceModel(proxy2)

        tv.setModel(model)
        lv.setModel(proxy1)
        lvf.setModel(proxy2)
        tvf.setModel(proxy3)

        splitter.addWidget(tv)
        splitter.addWidget(lv)
        splitter.addWidget(lvf)
        splitter.addWidget(tvf)


if __name__ == "__main__":
    import sys

    app = QApplication(sys.argv)
    w = Widget()
    w.show()
    sys.exit(app.exec())
